In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from IPython.display import display

In [3]:
# agregar X e Y

In [36]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [12]:
dir_path = '/content/drive/MyDrive/Dog Buster/Sandro'
dir_no_sandro = '/content/drive/MyDrive/Dog Buster/Sin Sandro'

In [44]:
# Arquitectura

model = models.Sequential()
model.add(layers.Conv2D(16, kernel_size=(6,6) , input_shape=(255,255,3), padding= 'valid', activation='relu'))
model.add(layers.MaxPool2D(pool_size=(3,3)))
model.add(layers.Conv2D(32, (5,5), activation='relu'))
model.add(layers.MaxPool2D(pool_size=(3,3)))

model.add(layers.Flatten())
model.add(layers.Dense(6, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))

model.summary()
optimizer = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['recall'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 250, 250, 16)      1744      
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 83, 83, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 79, 79, 32)        12832     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 26, 26, 32)        0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 21632)             0         
                                                                 
 dense_8 (Dense)             (None, 6)                

In [8]:
import os
from PIL import Image

In [9]:
def load_images_from_directory(directory):
    image_list = []

    # List all files in the directory
    files = os.listdir(directory)

    # Filter only files with certain image extensions (e.g., jpg, png)
    image_extensions = ['.jpg', '.jpeg']
    image_files = [file for file in files if os.path.splitext(file.lower())[1] in image_extensions]

    # Load each image and add it to the list
    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        image = Image.open(image_path)
        image_list.append(image)

    return image_list

In [10]:
img_vec = load_images_from_directory(dir_path)

In [21]:
img_vec_sin_sandro = load_images_from_directory(dir_no_sandro)

In [ ]:
display(img_vec[0])

In [54]:
X_sandro = [np.array(img_vec[i]) for i in range(0,10)]
X_sin_sandro = [np.array(img_vec_sin_sandro[i]) for i in range(len(img_vec_sin_sandro))]

In [58]:
X_sandro = np.array(X_sandro)/255
X_sin_sandro = np.array(X_sin_sandro)/255

In [59]:
X_sandro[0].shape

(2316, 3088, 3)

In [38]:
X = X_sandro + X_sin_sandro
len(X)

34

In [30]:
y_sandro = [1 for i in range(len(X_sandro))]
y_sandro

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [31]:
y_sin_sandro = [0 for i in range(len(X_sin_sandro))]

In [32]:
y_sin_sandro

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [35]:
y = y_sandro + y_sin_sandro

In [41]:
early_stopping = EarlyStopping(monitor='val_recall', patience=1, restore_best_weights=True)

In [ ]:
history = model.fit(x=X, y=y, batch_size=16, epochs=10, validation_split=0.2)